In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
19622,2024-12-09,Turquia Superliga,13:00,Galatasaray,Yalovaspor,1.10,6.53,165.5,1.87,1.86,-15.5,2.03,1.61,SzQfDvN0,0.909091,0.153139,0.534759,0.537634,0.062230,174.670,64.883386,0.371463,2.4,1.341641,0.559017,117.04,2.084,0.757450,0.363460,13.0,409.678,270.468844,0.660199,0.6,1.341641,2.236068,346.48,5.546,3.265996,0.588892,-37.0,76,71,1.54,4.88,129.690,330.808,1.006446,0.003791,0.163178,-0.81,-0.162,-0.617284,0.690995,0.5,-0.190995,-3.76,-0.752,-7.353723,0.596923,0.5,-0.096923
19623,2024-12-09,Suécia Liga De Basquete,15:00,Norrkoping F,Lulea F,6.56,1.06,147.5,1.83,1.83,8.5,1.97,1.71,C6lVUcc4,0.152439,0.943396,0.546448,0.546448,0.095835,138.450,88.509842,0.639291,1.8,1.643168,0.912871,68.34,1.726,0.932379,0.540196,19.0,921.420,1758.631177,1.908610,2.4,1.341641,0.559017,120.36,8.860,16.313021,1.841199,181.0,67,118,1.02,1.02,128.600,961.988,1.020758,0.000000,0.099917,-0.60,-0.120,-46.333333,0.755494,0.7,-0.055494,36.08,7.216,0.008315,0.550924,0.6,0.049076
19624,2024-12-09,Sérvia 1. Zls Feminino,14:30,Novosadska Novi Sad F,Ras F,2.66,1.40,135.5,1.85,1.81,-3.5,1.82,1.84,QmgFx1ig,0.375940,0.714286,0.540541,0.552486,0.090226,163.514,55.599713,0.340030,0.0,0.000000,NaN,252.88,2.720,1.050738,0.386301,-48.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,58,69,4.36,1.64,173.004,0.000,0.438894,0.015456,0.007728,-1.31,-0.262,-6.335878,0.484257,0.5,0.015743,0.00,0.000,inf,0.000000,0.0,0.000000
19625,2024-12-09,Rússia Liga Vtb United,14:00,Nizhny Novgorod,Samara,1.08,8.10,152.5,1.82,1.92,-16.5,2.01,1.68,xvoMu5vq,0.925926,0.123457,0.549451,0.520833,0.049383,183.984,85.182958,0.462991,1.8,1.643168,0.912871,125.49,2.366,1.422508,0.601229,16.0,385.718,356.040982,0.923060,1.2,1.643168,1.369306,329.04,5.306,4.537899,0.855239,-33.0,89,72,1.41,4.57,181.142,368.552,1.081457,0.037813,0.126474,-2.68,-0.536,-0.149254,0.564210,0.3,-0.264210,-2.40,-0.480,-14.791667,0.372755,0.3,-0.072755
19626,2024-12-09,Islândia Copa Da Islândia,16:30,Keflavik,Tindastoll,1.41,2.62,190.5,1.83,1.83,-6.5,1.99,1.70,dz2AwxmS,0.709220,0.381679,0.546448,0.546448,0.090899,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,92,79,2.15,1.65,0.000,0.000,0.424615,0.000000,0.111144,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
19627,2024-12-09,Romênia Divisão A,14:00,Rapid Bucuresti,Petrolul Ploiesti,1.02,8.80,156.5,1.86,1.81,-16.5,1.96,1.72,0riClB7r,0.980392,0.113636,0.537634,0.552486,0.094029,226.768,109.864313,0.484479,1.2,1.643168,1.369306,131.20,2.824,1.490278,0.527719,-6.0,295.344,126.043142,0.426767,0.0,0.000000,NaN,168.60,5.060,2.751545,0.543784,-141.0,80,60,1.64,2.81,199.328,251.834,1.120426,0.019267,0.092231,-2.26,-0.452,-0.044248,0.603369,0.5,-0.103369,-2.19,-0.438,-17.808219,0.487958,0.4,-0.087958
19628,2024-12-09,Israel Superliga,14:00,Gilboa Galil,Ironi Kiryat Ata,1.75,2.01,163.5,1.85,1.85,-2.5,1.98,1.71,Eu0NCZED,0.571429,0.497512,0.540541,0.540541,0.068941,269.940,158.232883,0.586178,0.0,0.000000,NaN,238.00,3.636,1.806497,0.496836,-83.0,334.070,290.816473,0.870526,1.8,1.643168,0.912871,82.56,3.638,2.993020,0.822710,31.0,56,64,4.25,1.29,0.000,508.690,0.097791,0.000000,0.103479,0.00,0.000,inf,0.00000

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
19637,21:00,Eua Ncaa,Norfolk State,Hofstra,1.69,Back Home
19665,08:35,China Cba,Shanxi Zhongyu,Beijing,1.88,Back Home
